In [1]:
import pandas as pd #导入Pandas
import numpy as np #导入Numpy
import jieba #导入结巴分词

from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU

In [2]:
neg = pd.read_excel('data/neg.xls', header=None, index=None)
pos = pd.read_excel('data/pos.xls', header=None, index=None)
neg['mark'] = 0
pos['mark'] = 1
pn = pd.concat([pos,neg],ignore_index=True)
#合并正负例
neglen = len(neg)
poslen = len(pos)
print(neglen, poslen)

10428 10677


In [3]:
cw = lambda x: list(jieba.cut(x))
pn['words'] = pn[0].apply(cw)
#apply函数：调用分词函数

comment = pd.read_excel('data/sum.xls')
comment = comment[comment['rateContent'].notnull()]
comment['words'] = comment['rateContent'].apply(cw)
# rateContent不为空则分词

Building prefix dict from the default dictionary ...
Loading model from cache /home/lilin/.local/lib/python3.7/site-packages/jieba/jieba.cache
Loading model cost 0.615 seconds.
Prefix dict has been built succesfully.


In [4]:
w2v_train = pd.concat([pn['words'],comment['words']],ignore_index=True)

w = []
for i in w2v_train:
    w.extend(i)

dict = pd.DataFrame(pd.Series(w).value_counts())
# print(dict[:10])
# 统计词频
del w,w2v_train
dict['id'] = list(range(1,len(dict)+1))

# print(dict['id'])

get_sent = lambda x: list(dict['id'][x])
pn['sent'] = pn['words'].apply(get_sent)
# 获取相应词id


In [5]:
maxlen = 50

print("Pad sequences (samples x time)")
pn['sent'] = list(sequence.pad_sequences(pn['sent'], maxlen=maxlen))
# 构建每个句子对应的单词id

x = np.array(list(pn['sent']))[::2] #训练集
y = np.array(list(pn['mark']))[::2]
xt = np.array(list(pn['sent']))[1::2] #测试集
yt = np.array(list(pn['mark']))[1::2]
xa = np.array(list(pn['sent'])) #全集
ya = np.array(list(pn['mark']))

Pad sequences (samples x time)


In [8]:
print('Build model...')
model = Sequential()
model.add(Embedding(len(dict)+1, 256))
model.add(LSTM(128)) # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x, y, batch_size=16, epochs=10) #训练时间为若干个小时


loss, acc = model.evaluate(xt,yt)
print('loss',loss)
print('accuary',acc)

Build model...
Epoch 1/10
660/660 [==============================] - 122s 185ms/step - loss: 0.4242 - accuracy: 0.8119
Epoch 2/10
660/660 [==============================] - 122s 185ms/step - loss: 0.1733 - accuracy: 0.9390
Epoch 3/10
660/660 [==============================] - 122s 185ms/step - loss: 0.0770 - accuracy: 0.9742
Epoch 4/10
660/660 [==============================] - 122s 185ms/step - loss: 0.0367 - accuracy: 0.9886
Epoch 5/10
660/660 [==============================] - 122s 185ms/step - loss: 0.0230 - accuracy: 0.9926
Epoch 6/10
660/660 [==============================] - 122s 185ms/step - loss: 0.0174 - accuracy: 0.9945
Epoch 7/10
660/660 [==============================] - 122s 185ms/step - loss: 0.0174 - accuracy: 0.9942
Epoch 8/10
660/660 [==============================] - 122s 185ms/step - loss: 0.0183 - accuracy: 0.9947
Epoch 9/10
660/660 [==============================] - 122s 185ms/step - loss: 0.0117 - accuracy: 0.9964
Epoch 10/10
330/330 [============================